+ http://www.acgeospatial.co.uk/julia-prt2/  
+ http://www.acgeospatial.co.uk/julia-prt3/

In [3]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

mkpath("downloads")
mkpath("out")

using ArchGDAL; const AG = ArchGDAL

 Activating environment at `C:\Users\i am\Documents\GitHub\stage-Machine-learning\prisma\hdf5\Project.toml`


ArchGDAL

In [1]:
using HDF5
include("../../code/julia/HDF5filesDict.jl")

Main.HDF5fd

In [9]:
include("../../code/julia/faux.jl")

Main.faux

# @GeoTiff

https://en.wikipedia.org/wiki/GeoTIFF  

https://en.wikipedia.org/wiki/TIFF  

https://en.wikipedia.org/wiki/Raster_graphics

tl;dr: abbiamo un raster (matrice di pixel) con metadati (numero bande, geolocation(lat, lon)), all interno di tiff, salvati tramite ArchGDAL -> GDAL, con i dati letti dai hdf prisma(cubi vnir+swir)

dove troviamo dati geoloc in prod l2d?  
* matrice lat: hdf5\swaths\PRS_L2d_(h per hypersp cube, p per panchromatic)CO\geolocation fields\latitude  
* matrice lon: hdf5\swaths\PRS_L2d_(h per hypersp cube, p per panchromatic)CO\geolocation fields\longitude

con le due mat abbiamo latitudine/longitudine di ogni pixel del raster

dove troviamo il cubo? (raster x bande)  
* cubo vnir:  hdf5\swaths\PRS_L2d_hCO\data fields\vnir_cube  
* cubo swir:  hdf5\swaths\PRS_L2d_hCO\data fields\swir_cube  


In [12]:
openfiles = HDF5fd.filesDict()
# 1 apriamo file hdf5
file = HDF5fd.open(openfiles, "data/PRS_L2D_STD_20200627102334_20200627102339_0001.he5", "r")



HDF5 data file: data/PRS_L2D_STD_20200627102334_20200627102339_0001.he5

In [35]:
# 2 prendiamo raster di una banda

cube = faux.getData(file, "HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/SWIR_Cube")

1235×173×1209 Array{UInt16,3}:
[:, :, 1] =
 0x0000  0x0000  0x0000  0x0000  0x0000  …  0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000  …  0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000  …  0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x000

In [36]:
# cubo X,band,Y, vogliamo una matrice X,Y per una banda
band = cube[:,130,:]

1235×1209 Array{UInt16,2}:
 0x0000  0x0000  0x0000  0x0000  0x0000  …  0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000  …  0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000  …  0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x0000
 0x0000  0x0000  0x0000  0x0000  0x0000     0x0000  0x0000  0x0000  0x000

In [39]:
#nb: sparse
count(!iszero,band)/(1235*1209)

0.5534429698985008

In [41]:
# 3 prendiamo matrici lat e lon
lat = faux.getData(file, "HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Latitude")
lon = faux.getData(file, "HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Longitude")

1235×1209 Array{Float64,2}:
 11.3955  11.3955  11.3955  11.3955  …  11.3819  11.3819  11.3819  11.3818
 11.3959  11.3959  11.3959  11.3959     11.3823  11.3822  11.3822  11.3822
 11.3963  11.3963  11.3963  11.3963     11.3826  11.3826  11.3826  11.3826
 11.3967  11.3967  11.3967  11.3967     11.383   11.383   11.383   11.383
 11.3971  11.3971  11.397   11.397      11.3834  11.3834  11.3834  11.3834
 11.3975  11.3974  11.3974  11.3974  …  11.3838  11.3838  11.3838  11.3837
 11.3978  11.3978  11.3978  11.3978     11.3842  11.3842  11.3841  11.3841
 11.3982  11.3982  11.3982  11.3982     11.3845  11.3845  11.3845  11.3845
 11.3986  11.3986  11.3986  11.3986     11.3849  11.3849  11.3849  11.3849
 11.399   11.399   11.399   11.399      11.3853  11.3853  11.3853  11.3853
 11.3994  11.3994  11.3993  11.3993  …  11.3857  11.3857  11.3857  11.3857
 11.3998  11.3997  11.3997  11.3997     11.3861  11.3861  11.386   11.386
 11.4001  11.4001  11.4001  11.4001     11.3865  11.3864  11.3864  11.3864

In [42]:
#possiamo chiudere il file
HDF5fd.closeall(openfiles)

chiuso 1 file


In [ ]:
# 4 convertiamo dati in un dataset

In [ ]:
# 4 scriviamo tiff
AG.write()

In [43]:
?ArchGDAL.write

```
write(dataset::AbstractDataset, filename::AbstractString; kwargs...)
```

Writes the dataset to the designated filename.
